In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import NASNetLarge
from keras.applications.nasnet import preprocess_input

In [ ]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 128
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [ ]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = NASNetLarge(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

In [ ]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [ ]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

In [ ]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [ ]:
model_prefix = 'nasnetlarge_raw_head_{}_lw{}'.format(size, lw)
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
]

In [ ]:
hists = []
hist = model.fit_generator(  
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

In [ ]:
model.load_weights('./models/{}.model'.format(model_prefix))

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

In [ ]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

In [ ]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)